In [1]:
import pandas as pd
from keras.models import model_from_yaml
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler            #importing all necessary libraries
from datetime import timedelta
from datetime import datetime

Using TensorFlow backend.


In [3]:
df=pd.read_csv('mac_1.csv')

In [4]:
yaml_file = open('model1.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model1.h5")
print("Loaded model from disk")                           #loading pre trained weights
            

Loaded model from disk


In [7]:
def forecastnext(ts):
    df['CPU1YBLPVDAKDLWAPP1'] = df['CPU1YBLPVDAKDLWAPP1'].apply(pd.to_datetime)
    b1=pd.to_datetime(ts)
    a1 = pd.to_datetime(ts) - timedelta(minutes=500)
    z=df[(df['CPU1YBLPVDAKDLWAPP1']>a1) & (df['CPU1YBLPVDAKDLWAPP1']<b1)]
    se=pd.DataFrame(z.x)
    size=se.size+1
    a2=[]
    x2=[]
    size1=size-51                                          
    series=se[size1:size]                                     #extract last 50 values from column
    scaler = MinMaxScaler(feature_range=(-1, 1))               
    scaled = scaler.fit_transform(series.values)
    series = pd.DataFrame(scaled)                             #scaling the values
    test_X=series.iloc[:]
    test_X=test_X.values
    test_X=test_X.reshape(1,50,1)
    n_future_preds=20                                    #number of predictions to be made
    preds_moving = []                                    # Use this to store the prediction made on each test window
    moving_test_window = [test_X[0,:].tolist()]          # Creating the first test window
    moving_test_window = np.array(moving_test_window)    # Making it an numpy array
    ts=pd.to_datetime(ts)
    tsac=ts.strftime('%Y-%m-%d %H:%M:%S')
    

    for i in range(n_future_preds):
        preds_one_step = loaded_model.predict(moving_test_window) # Note that this is already a scaled prediction so no need to rescale this
        preds_one_step.reshape(1,1)
        preds_one_step = scaler.inverse_transform(preds_one_step)                       #transforming into actual format

        preds_moving.append(preds_one_step[0,0]) # get the value from the numpy 2D array and append to predictions
        preds_one_step = preds_one_step.reshape(1,1,1) # Reshaping the prediction to 3D array for concatenation with moving test window
        moving_test_window = np.concatenate((moving_test_window[:,1:,:], preds_one_step), axis=1) # This is the new moving test window, where the first element from the window has been removed and the prediction  has been appended to the end
        ts=pd.to_datetime(ts)+timedelta(minutes=10)
        ts1=pd.to_datetime(ts)+timedelta(minutes=10)
        a2.append(ts.strftime('%Y-%m-%d %H:%M:%S'))
        x3=dict(zip(a2,preds_moving))
        x4=dict(zip('input',tsac))
        lst=[]
        lst.append({'input':tsac,'output':x3})
    return lst

In [8]:
forecastnext("2019-01-06 05:10:04")

[{'input': '2019-01-06 05:10:04',
  'output': {'2019-01-06 05:20:04': -83.28951,
   '2019-01-06 05:30:04': -292.1135,
   '2019-01-06 05:40:04': -509.79755,
   '2019-01-06 05:50:04': -723.85156,
   '2019-01-06 06:00:04': -931.1221,
   '2019-01-06 06:10:04': -1114.4348,
   '2019-01-06 06:20:04': -1240.0012,
   '2019-01-06 06:30:04': -1278.2535,
   '2019-01-06 06:40:04': -1229.6056,
   '2019-01-06 06:50:04': -1127.3274,
   '2019-01-06 07:00:04': -1008.18225,
   '2019-01-06 07:10:04': -897.0384,
   '2019-01-06 07:20:04': -805.748,
   '2019-01-06 07:30:04': -736.7186,
   '2019-01-06 07:40:04': -687.8455,
   '2019-01-06 07:50:04': -656.0063,
   '2019-01-06 08:00:04': -637.9409,
   '2019-01-06 08:10:04': -630.5801,
   '2019-01-06 08:20:04': -631.2353,
   '2019-01-06 08:30:04': -637.683}}]